#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 6, 14)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,22,B,TOP,"1,500",53.00,2,RD05pct,SET50,2022-02-02


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,BBL,2400,128.00,0,P50%,SET50,2022-04-22
1,4,B,GLOBAL,9000,20.50,1,3L,SET50,2022-02-02
2,5,B,GULF,6000,46.00,1,1L,SET50,2022-03-04
3,6,B,IVL,3000,48.00,1,P50%,SET50,2022-05-31
4,7,B,KBANK,1500,145.00,1,P50%,SET50,2022-04-18
5,13,B,KCE,1000,61.25,2,RD15pct,SET50,2022-03-21
6,8,B,KTC,2100,59.00,1,P25%,SET50,2022-04-18
7,22,B,TOP,1500,53.00,2,RD05pct,SET50,2022-02-02
8,12,S,BDMS,1500,26.25,1,CP1S,SET50,2022-03-09
9,20,S,TTB,100000,1.33,1,5pct,SET50,2022-04-21


In [6]:
name = 'GLOBAL'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,21.92,22.19,21.66,12248722.43,21.92
std,0.68,0.70,0.66,6329156.56,0.67
min,20.60,20.80,20.50,3132917.00,20.80
25%,21.48,21.70,21.20,7780736.75,21.40
50%,21.85,22.15,21.60,10654616.50,21.95
75%,22.42,22.72,22.30,15236992.00,22.40
max,23.20,23.70,23.00,38465202.00,23.40


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,BCPG,12000,11.1,1,MinP,SET100,2022-02-02
1,9,B,SINGER,3000,47.0,1,3L,SET100,2022-02-02
2,16,S,ORI,15000,11.1,0,CP1S,SET100,2022-05-09
3,17,S,RCL,4000,47.5,0,CP1S,SET100,2022-03-23


In [8]:
name = 'SINGER'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,52.32,53.23,51.46,4712521.63,52.28
std,3.08,2.95,2.95,3673773.31,3.07
min,47.00,48.25,46.75,1337690.00,46.75
25%,50.00,50.75,49.44,2439517.75,50.00
50%,51.88,53.00,51.12,3766360.00,51.88
75%,54.75,55.75,53.81,5631381.25,54.75
max,58.75,59.25,57.50,23670812.00,59.00


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,36.00,1,MinP,SET,2022-03-03
1,10,B,SSP,10000,9.35,1,6L,SET,2022-02-02
2,11,B,STANLY,600,177.00,1,P50%,SET,2022-02-02
3,15,S,NER,9000,7.10,0,P50%,SET,2022-04-19
4,18,S,SIS,3600,30.00,0,5pct,SET,2022-03-10


In [10]:
name = 'TMT'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,10.37,10.50,10.28,929588.70,10.41
std,0.72,0.74,0.69,1165494.22,0.72
min,9.10,9.20,9.05,34203.00,9.05
25%,10.10,10.20,10.00,257200.25,10.10
50%,10.30,10.40,10.20,517411.00,10.30
75%,11.00,11.10,10.80,1008813.25,10.92
max,11.90,12.00,11.60,7602048.00,11.90


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,36.00,1,MinP,SET,2022-03-03
1,2,B,BBL,2400,128.00,0,P50%,SET50,2022-04-22
2,3,B,BCPG,12000,11.10,1,MinP,SET100,2022-02-02
3,4,B,GLOBAL,9000,20.50,1,3L,SET50,2022-02-02
4,5,B,GULF,6000,46.00,1,1L,SET50,2022-03-04
5,6,B,IVL,3000,48.00,1,P50%,SET50,2022-05-31
6,7,B,KBANK,1500,145.00,1,P50%,SET50,2022-04-18
7,13,B,KCE,1000,61.25,2,RD15pct,SET50,2022-03-21
8,8,B,KTC,2100,59.00,1,P25%,SET50,2022-04-18
9,9,B,SINGER,3000,47.00,1,3L,SET100,2022-02-02
